In [5]:
import pandas as pd
from langchain_community.document_loaders import SeleniumURLLoader
from tqdm.auto import tqdm
import time
import os

In [3]:
# https://raw.githubusercontent.com/vladislavneon/RuBQ/master/RuBQ_2.0/RuBQ_2.0_test.json
df_test = pd.read_json('raw/RuBQ_2.0_test.json')
# https://raw.githubusercontent.com/vladislavneon/RuBQ/master/RuBQ_2.0/RuBQ_2.0_paragraphs.json
df_prg = pd.read_json('raw/RuBQ_2.0_paragraphs.json')

In [ ]:
df_rag = []
for idx, row in tqdm(df_test.iterrows(), total=df_test.shape[0]):
    try:
        if len(row['paragraphs_uids']['with_answer']) == 1:
            print(row['question_text'])
            print(row['answer_text'])
            p_id = row['paragraphs_uids']['with_answer'][0]
            prg = df_prg.query(f'uid == {p_id}')
            prg_text = prg['text'].values[0]
            prg_link = prg['ru_wiki_pageid'].values[0]
            print(prg_text)
            print('==================')
            print()
            
            fname = f'{prg_link}.txt'
            if not os.path.isfile(f"files/{fname}"):
                loader = SeleniumURLLoader(urls=[f'https://ru.wikipedia.org/?curid={prg_link}'])
                data = loader.load()
                content = data[0].page_content
                with open(f"files/{fname}", "w", encoding="utf-8") as f:
                    f.write(content)

            df_rag.append([row['question_text'], row['answer_text'], prg_text, f'{prg_link}.txt'])
    except:
        print(f'Ошибка: {idx}')
    
    time.sleep(1)
    
df_rag = pd.DataFrame(df_rag)
df_rag.columns = ['Вопрос','Правильный ответ','Контекст','Файл']

In [14]:
df_rag

,Вопрос,Правильный ответ,Контекст,Файл
0,Что может вызвать цунами?,Землетрясение,"Землетрясения, извержения вулканов и другие по...",19985.txt
1,Кто написал роман «Хижина дяди Тома»?,Г. Бичер-Стоу,«Хижина дяди Тома» (англ. Uncle Tom's Cabin) —...,273921.txt
2,Кто создал Зимний дворец в Санкт-Петербурге?,Растрелли,Зимний дворец (в 1918—1943 годах — Дворец Иску...,50.txt
3,Какое современное государство занимает террито...,Ирак,"Нине́ви́я или Нин (аккад. 𒌷𒉌𒉡𒀀, Ninwe; ассир. ...",36300.txt
4,Как звали отца Северуса Снейпа?,Тоббиас Снегг (Снейп),Происхождение Снегга в основном стало известно...,172300.txt
...,...,...,...,...
919,На какой реке стоит Череповец?,Шексна,Пятый гидроузел — Шекснинский (Череповецкий) Ш...,42745.txt
920,Какая страна является победителем Чемпионата м...,Германия,Магдале́на Но́йнер (нем. Magdalena Neuner; род...,400782.txt
921,Кто спроектировал Mercedes-AMG GT?,"Роберт Лесник, Марк Фетерстон, Ян Кауль",Автомобиль Mercedes-Benz SLS AMG имеет длину в...,1989102.txt
922,Какова высота Биг Бена?,96,"Высота башни, включая шпиль — 96 метров (315 ф...",329717.txt


In [15]:
df_rag.to_pickle('ru_rag_test_dataset.pkl')